# Programming onboard peripherals

----

## Contents

* [LEDs, switches and buttons](#LEDs, switches and buttons)
* [Controlling an LED](#Controlling an LED)
* [Example: Controlling all the LEDs, and buttons](#Example: Controlling all the LEDs and buttons)

----

## LEDs and buttons
The Microzed Carrier board has four blue LEDs (LD0-3) and 2 push-buttons (BTN1-2) that are connected to the Zynq’s programmable logic. 


Note that there are additional push-buttons and LEDs on the board, but these are used for specific functions (Power LED, PS reset button etc) and are not user accessible.

The IO can be controlled directly from Python. To demonstrate this, we first import the LED, Switch and Button classes from the module pynq.board:

In [5]:
from pynq.overlays.baseUZ import BaseUZOverlay
base = BaseUZOverlay("baseUZ.bit")

In [6]:
#?aquantis
from pprint import pprint

help(base)

Help on BaseUZOverlay in module pynq.overlays.baseUZ.baseUZ object:

class BaseUZOverlay(pynq.overlay.Overlay)
 |  The Aquantis overlay for the Microzed
 |  
 |  
 |  Attributes
 |  ----------
 |  iop1 : IOP
 |       IO processor connected to the PMODA interface
 |  iop2 : IOP
 |       IO processor connected to the PMODB interface
 |  trace_pmoda : pynq.logictools.TraceAnalyzer
 |      Trace analyzer block on PMODA interface, controlled by PS.
 |  trace_pmodb : pynq.logictools.TraceAnalyzer
 |      Trace analyzer block on PMODB interface, controlled by PS. . 
 |  leds : AxiGPIO
 |       4-bit output GPIO for interacting with the green LEDs LD0-3
 |  buttons : AxiGPIO
 |       2-bit input GPIO for interacting with the buttons BTN0-3
 |  
 |  Method resolution order:
 |      BaseUZOverlay
 |      pynq.overlay.Overlay
 |      pynq.pl.Bitstream
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, bitfile, **kwargs)
 |      Return a new Overlay object.
 |      
 | 

In [8]:
from pynq import GPIO
#help(GPIO)
from pynq import pl
#help(pl)
print("ip_dict")
pprint(base.ip_dict)
print("ps gpio_dict")
pprint(base.gpio_dict)
print("intterupt_pins")
pprint(base.interrupt_pins)
print("interrupt_controllers")
pprint(base.interrupt_controllers)
#help(x)

ip_dict
{'axi_ad9467_0': {'addr_range': 536870912,
                  'driver': <class 'pynq.overlay.DefaultIP'>,
                  'fullpath': 'axi_ad9467_0',
                  'gpio': {},
                  'interrupts': {},
                  'phys_addr': 1610612736,
                  'state': None,
                  'type': 'analog.com:user:axi_ad9467:1.0'},
 'axi_dmac_0': {'addr_range': 65536,
                'driver': <class 'pynq.overlay.DefaultIP'>,
                'fullpath': 'axi_dmac_0',
                'gpio': {},
                'interrupts': {'irq': {'controller': 'axi_intc_0',
                                       'fullpath': 'axi_dmac_0/irq',
                                       'index': 0}},
                'phys_addr': 1136787456,
                'state': None,
                'type': 'analog.com:user:axi_dmac:1.0'},
 'axi_gpio_0_button': {'addr_range': 65536,
                       'driver': <class 'pynq.lib.axigpio.AxiGPIO'>,
                       'fullpath': 'axi_

## Controlling an LED
Now we can create an instance of each of these classes and use their methods to manipulate them. Let’s start by instantiating a single LED and turning it on and off.

In [9]:
from pynq.lib import LED, Button

In [10]:
led0 = LED(0)

In [11]:
led0.on()

Check the board and confirm the LED is on.

In [12]:
led0.off()

Let’s then toggle _led0_ using the sleep() method from the _time_ package to see the LED flashing. 

In [13]:
#from pynq.board import LED, Button
import time
led0 = LED(0)
for i in range(20):
    led0.toggle()
    time.sleep(.1)

## Example: Controlling all the LEDs  and buttons


The example below creates 3 separate lists, called _leds_ and _buttons_.

In [14]:
# Set the number of Switches
MAX_LEDS =4
MAX_BUTTONS = 2

leds = [LED(index) for index in range(MAX_LEDS)]
buttons = [Button(index) for index in range(MAX_BUTTONS)] 




# Create lists for each of the IO component groups
for i in range(MAX_LEDS):
    leds[i] = LED(i)              
     
for i in range(MAX_BUTTONS):
    buttons[i] = Button(i) 

First, all LEDs are set to off. Then each switch is read, and if a switch is in the on position, the corresponding led is turned on. You can execute this cell a few times, changing the position of the switches on the board.

In [15]:
# LEDs start in the off state
for i in range(MAX_LEDS):
    leds[i].off()



The last part toggles the corresponding led (on or off) if a pushbutton is pressed. You can execute this cell a few times pressing different pushbuttons each time. 

In [17]:
# if a button is depressed, toggle the state of the corresponding LED
for i in range(MAX_LEDS):
    if buttons[i%2].read():
        leds[i].toggle()

We can do more with buttons cause an intterupt controller is attached. Let's wait till its pressed and released. As always we can check the help.

In [18]:
help(buttons[0])

Help on Button in module pynq.lib.button object:

class Button(builtins.object)
 |  This class controls the onboard push-buttons.
 |  
 |  Attributes
 |  ----------
 |  index : int
 |      Index of the push-buttons, starting from 0.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, index)
 |      Create a new Button object.
 |      
 |      Parameters
 |      ----------
 |      index : int
 |          The index of the push-buttons, from 0 to 1.
 |  
 |  read(self)
 |      Read the current value of the button.
 |      
 |      Returns
 |      -------
 |      int
 |          Either 1 if the button is pressed or 0 otherwise
 |  
 |  wait_for_value(self, value)
 |      Wait for the button to be pressed or released
 |      
 |      Parameters
 |      ----------
 |      value: int
 |          1 to wait for press or 0 to wait for release
 |      
 |      This function wraps the coroutine form so the asyncio
 |      event loop will run until the function returns
 |  
 |  wait_for_value_as

In [19]:
buttons[0].wait_for_value(1)
print("Button pressed")
buttons[0].wait_for_value(0)
print("Button released")

Button pressed
Button released


## Next steps

If you have time, write your own program to:
1. Turn on/off a single LED when a button is pressed
2. Shift the LED pattern when another button is pressed (the shift direction is determined by the value of the dip switch)
3. Toggle/Flash the LEDs for 5 seconds when another button is pressed
4. Change the delay between toggle when the last button is pressed. 